In [ ]:
import sys
import pandas as pd
from matplotlib import pyplot as plt
from dataclasses import dataclass
import numpy as np
from typing import Optional, List
from trade.utils import Position
from backtesting.backtest import BackTestResults
from loguru import logger
from common.utils import FeeRate

logger.remove()
logger.add(sys.stderr, level="ERROR")

%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ.update({"FINDATA": "/Users/andrybin/Yandex.Disk.localized/fin_data"})
os.environ.get("FINDATA", "fin_data")

In [ ]:
df = pd.read_csv("/Users/andrybin/Yandex.Disk.localized/Компьютер MINIMOZG/Files/DMA@LKOH-M15.txt")
# df["Volume"] = df.Type.abs()
# df.Type /= df.Volume
df.Time = list(map(np.datetime64, pd.to_datetime(df.Time)))
df

In [ ]:
from backtesting.backtest_broker import Broker
from common.type import Side


fee = FeeRate(0.008, 0)
wallet = 100000
leverage = 3
last_type = 0
last_pos = None                                 
positions = []
ticker = df.Symbol.iloc[0]

for i in range(0, len(df)):
    last_type = df.Type.iloc[i] + last_type
    if last_pos is not None:
        last_pos.close(df.Price.iloc[i], df.Time.iloc[i], i)
    if last_pos is not None:
        positions.append(last_pos)
    last_pos = None
    if last_type != 0:
        last_pos = Position(df.Price.iloc[i],
                            side=Side.from_int(last_type), 
                            date=np.datetime64(df.Time.iloc[i]), 
                            indx=i, 
                            ticker=ticker,
                            volume=wallet/df.Price.iloc[i]*leverage, 
                            fee_rate=fee)
        
br = Broker()
br.positions = positions

bres = BackTestResults("2015-01-01", "2024-10-10", wallet)
# bres.wallet = 100
bres.leverage = leverage
bres.process_backtest(br, leverage)

In [ ]:
from data_processing.dataloading import DataParser
from easydict import EasyDict
from common.type import TimePeriod


cfg = EasyDict({"data_type": "metatrader", "period": TimePeriod.M15, "ticker": "LKOH"})

data_pd, data_np = DataParser(cfg).load()
bres.compute_buy_and_hold(data_np.Date, data_np.Close, False)

In [ ]:
bres.monthly_hist

In [ ]:
bres.APR, bres.final_profit, bres.metrics, bres.fees

In [ ]:
bres.plot_results()

In [26]:
daily_hist_lkoh = bres.daily_hist

In [27]:
daily_hist_sr = bres.daily_hist

In [ ]:
plt.plot(daily_hist_lkoh.days, daily_hist_lkoh.profit, linewidth=1, color="b", alpha=0.6)
plt.plot(daily_hist_sr.days, daily_hist_sr.profit, linewidth=1, color="g", alpha=0.6)
profit_av = (daily_hist_sr.profit+daily_hist_lkoh.profit)/2
plt.plot(daily_hist_sr.days, profit_av, linewidth=3, color="r")

In [ ]:
APR_av, maxwait_av = bres.metrics_from_profit(profit_av.values)
APR_av, maxwait_av